In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/shsarv/Machine-Learning-Projects/main/Diabetes%20Prediction%20%5BEND%202%20END%5D/dataset/kaggle_diabetes.csv")

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,138,62,35,0,33.6,0.127,47,1
1,0,84,82,31,125,38.2,0.233,23,0
2,0,145,0,0,0,44.2,0.630,31,1
3,0,135,68,42,250,42.3,0.365,24,1
4,1,139,62,41,480,40.7,0.536,21,0
...,...,...,...,...,...,...,...,...,...
1995,2,75,64,24,55,29.7,0.370,33,0
1996,8,179,72,42,130,32.7,0.719,36,1
1997,6,85,78,0,0,31.2,0.382,42,0
1998,0,129,110,46,130,67.1,0.319,26,1


In [4]:
df.replace(0,np.nan,inplace=True)

In [5]:
df.isnull().sum()

Pregnancies                  301
Glucose                       13
BloodPressure                 90
SkinThickness                573
Insulin                      956
BMI                           28
DiabetesPedigreeFunction       0
Age                            0
Outcome                     1316
dtype: int64

In [6]:
df['Outcome'].replace(np.nan,0,inplace=True)
df['Pregnancies'].replace(np.nan,0,inplace=True)

In [8]:
t_c = ['BloodPressure','Glucose','SkinThickness','Insulin','BMI']

In [11]:
from sklearn.linear_model import LinearRegression
for i in t_c:
    temp_df = df[['Age',i]].dropna()
    lr = LinearRegression()
    lr.fit(temp_df[['Age']], temp_df[i])
    miss_df = df[['Age']][df[i].isnull()]
    df.loc[df[i].isnull(),i] = lr.predict(miss_df)


In [12]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [11]:
df['Pregnancies'].fillna(df['Pregnancies'].mode()[0],inplace=True)

In [33]:
df['BloodPressure'].fillna(df['BloodPressure'].mean(),inplace=True)

In [34]:
df['Glucose'].fillna(df['Glucose'].median(),inplace=True)
df['SkinThickness'].fillna(df['SkinThickness'].mean(),inplace=True)
df['Insulin'].fillna(df['Insulin'].mean(),inplace=True)
df['BMI'].fillna(df['BMI'].mean(),inplace=True)


In [35]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2.0,138.0,62.000000,35.000000,153.743295,33.6,0.127,47,1.0
1,0.0,84.0,82.000000,31.000000,125.000000,38.2,0.233,23,0.0
2,0.0,145.0,72.403665,29.341275,153.743295,44.2,0.630,31,1.0
3,0.0,135.0,68.000000,42.000000,250.000000,42.3,0.365,24,1.0
4,1.0,139.0,62.000000,41.000000,480.000000,40.7,0.536,21,0.0
...,...,...,...,...,...,...,...,...,...
1995,2.0,75.0,64.000000,24.000000,55.000000,29.7,0.370,33,0.0
1996,8.0,179.0,72.000000,42.000000,130.000000,32.7,0.719,36,1.0
1997,6.0,85.0,78.000000,29.341275,153.743295,31.2,0.382,42,0.0
1998,0.0,129.0,110.000000,46.000000,130.000000,67.1,0.319,26,1.0


In [36]:
y = df['Outcome']
X = df.drop("Outcome",axis = 1)

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.25,random_state = 42)

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [40]:
lgr = LogisticRegression(max_iter = 5000)
dr = DecisionTreeClassifier()

In [41]:
lgr.fit(X_train,y_train)
dr.fit(X_train,y_train)

DecisionTreeClassifier()

In [42]:
print(lgr.score(X_train,y_train),lgr.score(X_test,y_test))
print(dr.score(X_train,y_train),dr.score(X_test,y_test))

0.7626666666666667 0.784
1.0 0.966


In [43]:
from sklearn.model_selection import cross_val_score

In [44]:
cross_val_score(lgr,X_test,y_test,cv=5)

array([0.73, 0.82, 0.82, 0.76, 0.8 ])

In [45]:
from sklearn.model_selection import GridSearchCV

In [46]:
param_grid = {
    "C" : [0.01,0.1,1,10,100,1000],
    "penalty":['l2','l1','elasticnet'],
    'solver':['liblinear']
}

In [47]:
grid_search =GridSearchCV(estimator=lgr,param_grid=param_grid,cv=5,scoring='accuracy')
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
print("Best Parameters: ",best_params)

Best Parameters:  {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model

In [48]:
grid_search.score(X_test,y_test)

0.782

In [49]:
from sklearn.metrics import classification_report

In [50]:
print(classification_report(y_test,lgr.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.79      0.90      0.84       320
         1.0       0.76      0.58      0.66       180

    accuracy                           0.78       500
   macro avg       0.78      0.74      0.75       500
weighted avg       0.78      0.78      0.78       500



In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,lgr.predict(X_test))

array([[288,  32],
       [ 76, 104]])

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

print(confusion_matrix(y_test,lgr.predict(X_test)))
print(accuracy_score(y_test,lgr.predict(X_test)))
print(precision_score(y_test,lgr.predict(X_test)))
print(recall_score(y_test,lgr.predict(X_test)))
print(f1_score(y_test,lgr.predict(X_test)))

[[288  32]
 [ 76 104]]
0.784
0.7647058823529411
0.5777777777777777
0.6582278481012658


In [35]:
import pickle
with open('model.pickle','wb') as model_file:
    pickle.dump(lgr,model_file)